<a href="https://colab.research.google.com/github/Rishi-R-h150/SpamorHam/blob/main/spamdetection_ipnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
filename="/content/archive (4).zip"
import zipfile
zip_ref = zipfile.ZipFile(filename, "r")
zip_ref.extractall()
zip_ref.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/archive (4).zip'

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/spam_or_not_spam.csv")

In [ ]:
train_df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [ ]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,email,label
4336,this is possible however using sa as a block c...,0
584,the usual crud why do morons ranting and beati...,0
240,near the end of his memoirs of extraordinary p...,0
373,on sun NUMBER sep NUMBER joseph s barrera iii ...,0
5207,take control of your computer with this top o...,1


In [ ]:
train_df.label.value_counts()

0    4331
1    1000
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train_sentences,val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled["email"].to_numpy(),
                                                                         train_df_shuffled["label"].to_numpy(),
                                                                          test_size=0.1,
                                                                         random_state=42
                                                                         )

In [ ]:
train_sentences[:5]

array([' date fri NUMBER sep NUMBER NUMBER NUMBER NUMBER NUMBER from tony nugent tony linuxworks com au message id NUMBER gNUMBERdNUMBERqufNUMBER hobbit linuxworks com au nospam i can cut n paste from exmh s message display window into spawned gvim processes but not into anything else that s odd i cut paste between all kinds of windows exmh into mozilla xterm another wish script of mine i use for dns tasks but that one i guess is to be expected netscape when i used to use it but i suppose it and mozilla are the same codebase approx in fact i can t thing of anything it fails for that i have noticed what is an example of an anything else that it fails for for you kre _______________________________________________ exmh users mailing list exmh users URL URL ',
       ' gary funck said i thought the perllocal pod line looked odd is it normal to write documentation into the root directory g is there some make parameter o r environment variable that should ve been set when i ran make an issu

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(4797, 4797, 534, 534)

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

AttributeError: ignored

In [ ]:
# Assuming train_sentences is a list of sentences
# For example: train_sentences = ["This is a sentence.", "Another example sentence.", "And one more.", "Yet another.",  "Short one."]

# Filter out any non-string elements (e.g., floats)
valid_sentences = [i for i in train_sentences if isinstance(i, str)]

# Calculate the length of each sentence in terms of words and create a list
sentence_lengths = [len(sentence.split()) for sentence in valid_sentences]

# Sum up the lengths of all sentences
total_word_count = sum(sentence_lengths)

# Calculate the average number of words per sentence
average_words_per_sentence = total_word_count / len(valid_sentences)

# Round the result to the nearest whole number
rounded_average = round(average_words_per_sentence)

# Print or use the rounded average as needed
print(rounded_average)


226


In [ ]:
max_vocab_length = 10000
max_length = rounded_average

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    standardize = "lower_and_strip_punctuation",
                                    output_mode = "int",
                                    output_sequence_length = max_length
                                    )

In [ ]:
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
train_sentences = [str(sentence) for sentence in train_sentences]
text_vectorizer.adapt(np.array(train_sentences))

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length,
                             name="embedding_1")

embedding

In [ ]:
import tensorflow_hub as hub

In [ ]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [ ]:
model_0 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1, activation="sigmoid")
],name="USE_model")

In [ ]:
model_0.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_0.summary()

Model: "USE_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [ ]:
train_sentences= np.array(train_sentences)

In [ ]:
model_0_history = model_0.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
)

Epoch 1/10
150/150 [==============================] - 18s 100ms/step - loss: 0.2508 - accuracy: 0.9110 - val_loss: 0.0870 - val_accuracy: 0.9850
Epoch 2/10
150/150 [==============================] - 13s 86ms/step - loss: 0.0603 - accuracy: 0.9846 - val_loss: 0.0419 - val_accuracy: 0.9888
Epoch 3/10
150/150 [==============================] - 12s 82ms/step - loss: 0.0360 - accuracy: 0.9894 - val_loss: 0.0291 - val_accuracy: 0.9906
Epoch 4/10
150/150 [==============================] - 11s 75ms/step - loss: 0.0261 - accuracy: 0.9925 - val_loss: 0.0217 - val_accuracy: 0.9944
Epoch 5/10
150/150 [==============================] - 12s 80ms/step - loss: 0.0199 - accuracy: 0.9952 - val_loss: 0.0178 - val_accuracy: 0.9925
Epoch 6/10
150/150 [==============================] - 13s 85ms/step - loss: 0.0163 - accuracy: 0.9956 - val_loss: 0.0148 - val_accuracy: 0.9963
Epoch 7/10
150/150 [==============================] - 14s 93ms/step - loss: 0.0129 - accuracy: 0.9973 - val_loss: 0.0133 - val_accuracy

In [ ]:
test_sample_1=[" hyperlink hyperlink hyperlink let mortgage lenders compete for your business did you receive an email advertisement in error our goal is to only target individuals who would like to take advantage of our offers if you d like to be removed from our mailing list please click on the link below you will be removed immediately and automatically from all of our future mailings we protect all email addresses from other third parties thank you hyperlink please remove me "]

In [ ]:
val_sentences[:5]

array([' gary lawrence murphy wrote and say hello to the cool oooo this is going to cause some stir of course not some people just don t want to be confused by the facts for anyone to fully bury global warming they would need to explain why the dramatic increase in coNUMBER concentrations are not increasing the global temperature they would also need to explain why worldwide glaciers are melting faster than they have previously in the historical record that is people need more than refutations they need a compelling alternate explanation hint climate variability doesn t cover all the bases jim ',
       ' URL additional comments from easmith URL NUMBER NUMBER NUMBER NUMBER NUMBER i like this idea but from my reading part of my dissertation research on gas or in this case more precisely evolutionary programming since the parameters are non binary and the major operator is mutation not crossover you do need to keep in mind that the more interactions there are between different variables 

In [ ]:
test_sample=["with all due respect you are fired"]

In [ ]:
pred_prob = tf.squeeze(model_0.predict([test_sample]))
pred = tf.round(pred_prob)

1/1 [==============================] - 0s 54ms/step


In [ ]:
if pred==1:
  print("It is a spam")
else:
  print("Not a spam")

Not a spam


In [ ]:
train_df.label.value_counts()

0    4331
1    1000
Name: label, dtype: int64